### Installing dependencies

In [ ]:
!pip install openai-agents

### Import OpenAI API Key

In [ ]:
import os
from openai import OpenAI

# API Keys and Credentials
os.environ["OPENAI_API_KEY"] = ""

### Create your first Agent!

In [ ]:
from agents import Agent

# Create an agent
agent = Agent(
    name="Assistant", 
    instructions="You are a helpful assistant",
    model="gpt-4o-mini"
)

### Run your Agent!

In [ ]:
from agents import Runner

# Run the agent
result = await Runner.run(
    agent, 
    "Write a haiku about recursion in programming."
)

print(result.final_output)

### Use Tools with Agents!

In [ ]:
from agents import WebSearchTool

browser_agent = Agent(
    name="Browser Agent", 
    instructions="You can browse the web for information.",
    model="gpt-4o-mini",
    tools=[WebSearchTool()]
)

result = await Runner.run(
    browser_agent, 
    "What is today's date?",
)

print(result.final_output)

### Can we use Agents as Tools for Other Agents?

In [ ]:
# Specialized agents
malayalam_agent = Agent(
    name="Malayalam agent",
    instructions="You translate the user's message to malayalam. End the sentence with hashtags.",
)

tamil_agent = Agent(
    name="Tamil agent",
    instructions="You translate the user's message to Tamil. End the sentence with hashtags.",
)

# Orchestrator agent that uses other agents as tools
orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate. "
        "If asked for multiple translations, you call the relevant tools."
    ),
    tools=[
        malayalam_agent.as_tool(
            tool_name="translate_to_malayalam",
            tool_description="Translate the user's message to malayalam.",
        ),
        tamil_agent.as_tool(
            tool_name="translate_to_tamil",
            tool_description="Translate the user's message to tamil",
        ),
    ],
)

result = await Runner.run(
    orchestrator_agent, 
    "Say 'Hello, how are you?' in malayalam and tamil."
)
print(result.final_output)

### Handoff Agents!

In [ ]:
from agents import Agent, handoff

amma = Agent(
    name="Amma",
    instructions="You are the mother. Present your parental role in a nonchalant manner and act like a mother.",
    model="gpt-4o"
)
achan = Agent(
    name="Achan",
    instructions="You are the father. Present your parental role in a nonchalant manner and act like a father.",
    model="gpt-4o"
)

In [ ]:
triage_agent = Agent(name="Triage agent", instructions= "You have to handoff to the father if it is a serious case, if it is sensitive handoff to mother", handoffs=[amma, achan])

In [ ]:
# Run the agent
result = await Runner.run(
    triage_agent, 
    "I need money!"    
)

print(result.final_output)

### MCP Tool Calling

In [ ]:
# Simple GitMCP Tool
from agents import HostedMCPTool

agent = Agent(
    name="Assistant",
    tools=[
        HostedMCPTool(
            tool_config={
                "type": "mcp",
                "server_label": "gitmcp",
                "server_url": "https://gitmcp.io/openai/codex",
                "require_approval": "never",
            }
        )
    ],
)

result = await Runner.run(agent, "Which language is this repository written in: https://github.com/aloshdenny/superfloat")
print(result.final_output)